In [216]:
import mysql.connector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [217]:
rds_host='mybike.c0jxuz6r8olg.us-west-2.rds.amazonaws.com'
name='hibike'
pwd='zx123456'
db_name='bike'
port=3306

conn = mysql.connector.connect(host=rds_host,user=name, password=pwd, database=db_name)

df = pd.read_sql("select * from weather", con=conn)
df1 = pd.read_sql("select * from station_1", con=conn)
df.shape

(249, 6)

In [218]:
df1.tail(10)

last_update status  bike_stands  available_bike_stands  \
717 2018-04-18 13:07:32   OPEN           31                      7   
718 2018-04-18 13:10:33   OPEN           31                      7   
719 2018-04-18 13:13:28   OPEN           31                      9   
720 2018-04-18 13:16:35   OPEN           31                      9   
721 2018-04-18 13:17:26   OPEN           31                      7   
722 2018-04-18 13:21:25   OPEN           31                      8   
723 2018-04-18 13:24:16   OPEN           31                      9   
724 2018-04-18 13:29:23   OPEN           31                      9   
725 2018-04-18 13:31:59   OPEN           31                      9   
726 2018-04-18 13:35:35   OPEN           31                      8   

     available_bikes  
717               24  
718               24  
719               22  
720               22  
721               24  
722               23  
723               22  
724               22  
725               22  
726               23

In [219]:
df['month'] = df['last_update'].dt.month
df['day'] = df['last_update'].dt.day
df['weekday'] = df['last_update'].dt.weekday_name
df['hour'] = df['last_update'].dt.hour
df['minute'] = df['last_update'].dt.minute

df1['month'] = df1['last_update'].dt.month
df1['day'] = df1['last_update'].dt.day
df1['weekday'] = df1['last_update'].dt.weekday_name
df1['hour'] = df1['last_update'].dt.hour
df1['minute'] = df1['last_update'].dt.minute

In [220]:
df1.minute = (df1.minute > 30) *30

In [221]:
df2 = pd.merge(df, df1, left_on=['minute', 'hour', 'weekday', 'month', 'day'], right_on=['minute', 'hour', 'weekday', 'month', 'day'])
df2

last_update_x weather_main humidity    temp  visibility  wind  \
0   2018-04-13 01:00:00      Drizzle       93  280.15      6000.0   4.1   
1   2018-04-13 01:00:00      Drizzle       93  280.15      6000.0   4.1   
2   2018-04-13 01:30:00      Drizzle       93  280.15      7000.0   3.1   
3   2018-04-13 01:30:00      Drizzle       93  280.15      7000.0   3.1   
4   2018-04-13 01:30:00      Drizzle       93  280.15      7000.0   3.1   
5   2018-04-13 02:00:00      Drizzle       93  280.15      7000.0   3.6   
6   2018-04-13 02:00:00      Drizzle       93  280.15      7000.0   3.6   
7   2018-04-13 02:00:00      Drizzle       93  280.15      7000.0   3.6   
8   2018-04-13 02:30:00      Drizzle       93  280.15      7000.0   3.6   
9   2018-04-13 02:30:00      Drizzle       93  280.15      7000.0   3.6   
10  2018-04-13 02:30:00      Drizzle       93  280.15      7000.0   3.6   
11  2018-04-13 03:00:00      Drizzle       93  280.15      6000.0   3.1   
12  2018-04-13 03:00:00      Drizzle       93  280.15      6000.0   3.1   
13  2018-04-13 03:00:00      Drizzle       93  280.15      6000.0   3.1   
14  2018-04-13 03:30:00      Drizzle      100  279.72      6000.0   3.1   
15  2018-04-13 03:30:00      Drizzle      100  279.72      6000.0   3.1   
16  2018-04-13 03:30:00      Drizzle      100  279.72      6000.0   3.1   
17  2018-04-13 04:00:00      Drizzle      100  279.66      6000.0   3.1   
18  2018-04-13 04:00:00      Drizzle      100  279.66      6000.0   3.1   
19  2018-04-13 04:00:00      Drizzle      100  279.66      6000.0   3.1   
20  2018-04-13 05:00:00      Drizzle      100  279.77      7000.0   2.6   
21  2018-04-13 05:00:00      Drizzle      100  279.77      7000.0   2.6   
22  2018-04-13 05:00:00      Drizzle      100  279.77      7000.0   2.6   
23  2018-04-13 05:00:00      Drizzle      100  279.77      7000.0   2.6   
24  2018-04-13 05:30:00      Drizzle      100  279.71      5000.0   3.1   
25  2018-04-13 05:30:00      Drizzle      100  279.71      5000.0   3.1   
26  2018-04-13 05:30:00      Drizzle      100  279.71      5000.0   3.1   
27  2018-04-13 05:30:00      Drizzle      100  279.71      5000.0   3.1   
28  2018-04-13 06:00:00      Drizzle      100  279.77      5000.0   3.6   
29  2018-04-13 06:00:00      Drizzle      100  279.77      5000.0   3.6   
..                  ...          ...      ...     ...         ...   ...   
634 2018-04-18 10:30:00       Clouds       82  286.66     10000.0   6.7   
635 2018-04-18 10:30:00       Clouds       82  286.66     10000.0   6.7   
636 2018-04-18 10:30:00       Clouds       82  286.66     10000.0   6.7   
637 2018-04-18 11:00:00      Drizzle       82  287.16     10000.0   7.2   
638 2018-04-18 11:00:00      Drizzle       82  287.16     10000.0   7.2   
639 2018-04-18 11:00:00      Drizzle       82  287.16     10000.0   7.2   
640 2018-04-18 11:00:00      Drizzle       82  287.16     10000.0   7.2   
641 2018-04-18 11:30:00      Drizzle       87  287.18     10000.0   7.7   
642 2018-04-18 11:30:00      Drizzle       87  287.18     10000.0   7.7   
643 2018-04-18 11:30:00      Drizzle       87  287.18     10000.0   7.7   
644 2018-04-18 11:30:00      Drizzle       87  287.18     10000.0   7.7   
645 2018-04-18 12:00:00       Clouds       87  288.18     10000.0   8.7   
646 2018-04-18 12:00:00       Clouds       87  288.18     10000.0   8.7   
647 2018-04-18 12:00:00       Clouds       87  288.18     10000.0   8.7   
648 2018-04-18 12:00:00       Clouds       87  288.18     10000.0   8.7   
649 2018-04-18 12:00:00       Clouds       87  288.18     10000.0   8.7   
650 2018-04-18 12:00:00       Clouds       87  288.18     10000.0   8.7   
651 2018-04-18 12:00:00       Clouds       87  288.18     10000.0   8.7   
652 2018-04-18 12:00:00       Clouds       87  288.18     10000.0   8.7   
653 2018-04-18 12:00:00       Clouds       87  288.18     10000.0   8.7   
654 2018-04-18 12:30:00       Clouds       82  288.16     10000.0   8.2   
655 2018-04-18 12:30:00       Cloud

In [222]:
df2 = df2.drop(['last_update_x'] , axis=1)

In [223]:
df2['minute'] = df2['last_update_y'].dt.minute + df2['last_update_y'].dt.hour * 60
df2['minute']
pre_minute = df2['minute'][:-1].tolist()
pre_minute.insert(0, df2['minute'][0])
df2['pre_minute'] = pre_minute
continuous_columns.append('pre_minute')
df2

weather_main humidity    temp  visibility  wind  month  day    weekday  \
0        Drizzle       93  280.15      6000.0   4.1      4   13     Friday   
1        Drizzle       93  280.15      6000.0   4.1      4   13     Friday   
2        Drizzle       93  280.15      7000.0   3.1      4   13     Friday   
3        Drizzle       93  280.15      7000.0   3.1      4   13     Friday   
4        Drizzle       93  280.15      7000.0   3.1      4   13     Friday   
5        Drizzle       93  280.15      7000.0   3.6      4   13     Friday   
6        Drizzle       93  280.15      7000.0   3.6      4   13     Friday   
7        Drizzle       93  280.15      7000.0   3.6      4   13     Friday   
8        Drizzle       93  280.15      7000.0   3.6      4   13     Friday   
9        Drizzle       93  280.15      7000.0   3.6      4   13     Friday   
10       Drizzle       93  280.15      7000.0   3.6      4   13     Friday   
11       Drizzle       93  280.15      6000.0   3.1      4   13     Friday   
12       Drizzle       93  280.15      6000.0   3.1      4   13     Friday   
13       Drizzle       93  280.15      6000.0   3.1      4   13     Friday   
14       Drizzle      100  279.72      6000.0   3.1      4   13     Friday   
15       Drizzle      100  279.72      6000.0   3.1      4   13     Friday   
16       Drizzle      100  279.72      6000.0   3.1      4   13     Friday   
17       Drizzle      100  279.66      6000.0   3.1      4   13     Friday   
18       Drizzle      100  279.66      6000.0   3.1      4   13     Friday   
19       Drizzle      100  279.66      6000.0   3.1      4   13     Friday   
20       Drizzle      100  279.77      7000.0   2.6      4   13     Friday   
21       Drizzle      100  279.77      7000.0   2.6      4   13     Friday   
22       Drizzle      100  279.77      7000.0   2.6      4   13     Friday   
23       Drizzle      100  279.77      7000.0   2.6      4   13     Friday   
24       Drizzle      100  279.71      5000.0   3.1      4   13     Friday   
25       Drizzle      100  279.71      5000.0   3.1      4   13     Friday   
26       Drizzle      100  279.71      5000.0   3.1      4   13     Friday   
27       Drizzle      100  279.71      5000.0   3.1      4   13     Friday   
28       Drizzle      100  279.77      5000.0   3.6      4   13     Friday   
29       Drizzle      100  279.77      5000.0   3.6      4   13     Friday   
..           ...      ...     ...         ...   ...    ...  ...        ...   
634       Clouds       82  286.66     10000.0   6.7      4   18  Wednesday   
635       Clouds       82  286.66     10000.0   6.7      4   18  Wednesday   
636       Clouds       82  286.66     10000.0   6.7      4   18  Wednesday   
637      Drizzle       82  287.16     10000.0   7.2      4   18  Wednesday   
638      Drizzle       82  287.16     10000.0   7.2      4   18  Wednesday   
639      Drizzle       82  287.16     10000.0   7.2      4   18  Wednesday   
640      Drizzle       82  287.16     10000.0   7.2      4   18  Wednesday   
641      Drizzle       87  287.18     10000.0   7.7      4   18  Wednesday   
642      Drizzle       87  287.18     10000.0   7.7      4   18  Wednesday   
643      Drizzle       87  287.18     10000.0   7.7      4   18  Wednesday   
644      Drizzle       87  287.18     10000.0   7.7      4   18  Wednesday   
645       Clouds       87  288.18     10000.0   8.7      4   18  Wednesday   
646       Clouds       87  288.18     10000.0   8.7      4   18  Wednesday   
647       Clouds       87  288.18     10000.0   8.7      4   18  Wednesday   
648       Clouds       87  288.18     10000.0   8.7      4   18  Wednesday   
649       Clouds       87  288.18     10000.0   8.7      4   18  Wednesday   
650       Clouds       87  288.18     10000.0   8.7      4   18  Wednesday   
651       Clouds       87  288.18     10000.0   8.7      4   18  Wednesday   
652       Clouds       87  288.18     10000.0   8.7      4   18  Wednesday   
653       Clouds       87 

In [224]:
categorical_columns = df2[['weather_main', 'weekday', 'status']]
for column in categorical_columns:
    df2[column] = df2[column].astype('category')
df2.select_dtypes(['category']).describe().T

count unique     top freq
weather_main   664      6  Clouds  379
weekday        664      6  Friday  163
status         664      1    OPEN  664

In [225]:
# Select columns containing continuous data
continuous_columns = []
for column in df2.columns.values:
    if column not in categorical_columns:
        continuous_columns.append(column)
df2['humidity'] = df2['humidity'].astype('float')

continuous_columns.pop(continuous_columns.index('last_update_y'))
print(df2[continuous_columns].dtypes)

humidity                 float64
temp                     float64
visibility               float64
wind                     float64
month                      int64
day                        int64
hour                       int64
minute                     int64
bike_stands                int64
available_bike_stands      int64
available_bikes            int64
pre_minute                 int64
dtype: object


In [226]:
df2.isnull().sum()

weather_main             0
humidity                 0
temp                     0
visibility               0
wind                     0
month                    0
day                      0
weekday                  0
hour                     0
minute                   0
last_update_y            0
status                   0
bike_stands              0
available_bike_stands    0
available_bikes          0
pre_minute               0
dtype: int64

In [227]:
# Check for irregular cardinality in categorical features. There could be same values spelled differently
for column in categorical_columns:
    print("Unique values for " + column + ":", pd.unique(df2[column].ravel()))

Unique values for weather_main: ['Drizzle' 'Mist' 'Clouds' 'Fog' 'Clear' 'Rain']
Unique values for weekday: ['Friday' 'Saturday' 'Sunday' 'Monday' 'Tuesday' 'Wednesday']
Unique values for status: ['OPEN']


In [228]:
#df2['pre_available_bikes'][0] = df2['available_bikes'][0]
#df2['pre_available_bikes'].iloc[1:] = df2['available_bikes'][:-1]
#pd.concat([df2['available_bikes'][0:1], df2['available_bikes'][:-1]])
pre_available_bikes = df2['available_bikes'][:-1].tolist()
pre_available_bikes.insert(0, df2['available_bikes'][0])
df2['pre_available_bikes'] = pre_available_bikes
continuous_columns.append('pre_available_bikes')

In [229]:
df_dummies_weather_main = pd.get_dummies(df2[['weather_main']])
print(df_dummies_weather_main.head())
df_dummies_weekday = pd.get_dummies(df2[['weekday']])
df_dummies_weekday.head()

   weather_main_Clear  weather_main_Clouds  weather_main_Drizzle  \
0                   0                    0                     1   
1                   0                    0                     1   
2                   0                    0                     1   
3                   0                    0                     1   
4                   0                    0                     1   

   weather_main_Fog  weather_main_Mist  weather_main_Rain  
0                 0                  0                  0  
1                 0                  0                  0  
2                 0                  0                  0  
3                 0                  0                  0  
4                 0                  0                  0  


weekday_Friday  weekday_Monday  weekday_Saturday  weekday_Sunday  \
0               1               0                 0               0   
1               1               0                 0               0   
2               1               0                 0               0   
3               1               0                 0               0   
4               1               0                 0               0   

   weekday_Tuesday  weekday_Wednesday  
0                0                  0  
1                0                  0  
2                0                  0  
3                0                  0  
4                0                  0

In [230]:
describe_columns = [x for x in continuous_columns if x not in ['available_bike_stands', 'bike_stands', 'available_bikes', 'month', 'visibility']]

In [231]:
X = pd.concat([df2[describe_columns], df_dummies_weather_main, df_dummies_weekday], axis =1)
y = df2.available_bikes

In [232]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline
print(continuous_columns)

['humidity', 'temp', 'visibility', 'wind', 'month', 'day', 'hour', 'minute', 'bike_stands', 'available_bike_stands', 'available_bikes', 'pre_minute', 'pre_available_bikes']


In [233]:
#high temp  high wind , hour

In [234]:
#dtc = DecisionTreeClassifier(max_depth=10, random_state=1)
#dtc.fit(X, y)
#print(dtc)

In [235]:
#from sklearn.cross_validation import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7)

In [236]:
#type(dtc)

In [237]:
# Compute the importance of each feature based on the trained decision tree classifier
#pd.DataFrame({'feature': X.columns, 'importance': dtc.feature_importances_})

import operator

import operator

def tree_print(clf, X):
    """
    Print the tree of a sklearn DecisionTreeClassifier

    Parameters
    ----------
    clf : DecisionTreeClassifier - A tree that has already been fit.
    X : The original training set
    """
    tlevel = _tree_rprint('', clf, X.columns, clf.classes_)
    print('<',end='')
    for i in range(3*tlevel - 2):
        print('-',end='')
    print('>')
    print('Tree Depth: ',tlevel)

def _tree_rprint(kword, clf, features, labels, node_index=0, tlevel_index=0):
    # Note: The DecisionTreeClassifier uses the Tree structure defined in:
    # 		github.com/scikit-learn/scikit-learn/blob/master/sklearn/tree/_tree.pyx
    #       it is an array based tree implementation:

    # indent the nodes according to their tree level
    for i in range(tlevel_index):
        print('  |',end='')

	#  TODO: the following should use the TREE_LEAF constant defined in _tree.pyx
	#        instead of -1, not quite sure how to get at it from the tree user level
    if clf.tree_.children_left[node_index] == -1:  # indicates leaf
        print(kword, end=' ' if kword else '')
        # get the majority label
        count_list = clf.tree_.value[node_index, 0]
        max_index, max_value = max(enumerate(count_list), key=operator.itemgetter(1))
        max_label = labels[max_index]
        print(max_label)
        return tlevel_index
    
    else:
        # compute and print node label
        feature = features[clf.tree_.feature[node_index]]
        threshold = clf.tree_.threshold[node_index]
        print(kword, end=' ' if kword else '')
        print('if {} =< {}: '.format(feature, threshold))
        # recurse down the children
        left_index = clf.tree_.children_left[node_index]
        right_index = clf.tree_.children_right[node_index]
        ltlevel_index = _tree_rprint('then', clf, features, labels, left_index, tlevel_index+1)
        rtlevel_index = _tree_rprint('else', clf, features, labels, right_index, tlevel_index+1)
        # return the maximum depth of either one of the children
        return max(ltlevel_index,rtlevel_index)
tree_print(dtc, X)

In [238]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

print("Training data:\n", pd.concat([X_train, y_train], axis=1))
print("\nTest data:\n", pd.concat([X_test, y_test], axis=1))

Training data:
      humidity    temp  wind  day  hour  minute  pre_minute  \
540      81.0  284.15   4.6   18     0      42          31   
645      87.0  288.18   8.7   18    12     721         708   
388      93.0  282.66  10.8   15    12     728         717   
211      81.0  284.15   5.7   14     9     545         535   
40      100.0  279.15   3.6   13     7     460         454   
503      66.0  286.15  11.8   17    19    1181        1177   
402      93.0  283.16   9.8   15    14     840         833   
3        93.0  280.15   3.1   13     1     103          93   
243      81.0  285.29   7.7   14    13     820         812   
161     100.0  281.28   1.5   13    23    1427        1420   
387      93.0  282.73  10.3   15    11     717         707   
295      87.0  283.16   5.1   14    21    1263        1257   
333      93.0  281.72   6.7   15     2     153         143   
158     100.0  281.16   3.1   13    23    1398        1392   
142      93.0  280.66   3.1   13    21    1266        

In [239]:
# Train RF with 100 trees
rfc = RandomForestClassifier(n_estimators=100, max_features='auto', oob_score=True, random_state=1)
rfc.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=True, random_state=1, verbose=0, warm_start=False)

In [240]:
pd.DataFrame({'feature': X.columns, 'importance':rfc.feature_importances_})

feature  importance
0               humidity    0.032209
1                   temp    0.073271
2                   wind    0.060839
3                    day    0.041120
4                   hour    0.065661
5                 minute    0.194565
6             pre_minute    0.202946
7    pre_available_bikes    0.227021
8     weather_main_Clear    0.008829
9    weather_main_Clouds    0.011222
10  weather_main_Drizzle    0.012047
11      weather_main_Fog    0.001285
12     weather_main_Mist    0.002311
13     weather_main_Rain    0.006432
14        weekday_Friday    0.011627
15        weekday_Monday    0.003266
16      weekday_Saturday    0.018020
17        weekday_Sunday    0.017457
18       weekday_Tuesday    0.004066
19     weekday_Wednesday    0.005808

In [241]:
rfc.predict_proba(X)
print(X.loc[1])

humidity                 93.00
temp                    280.15
wind                      4.10
day                      13.00
hour                      1.00
minute                   82.00
pre_minute               72.00
pre_available_bikes       1.00
weather_main_Clear        0.00
weather_main_Clouds       0.00
weather_main_Drizzle      1.00
weather_main_Fog          0.00
weather_main_Mist         0.00
weather_main_Rain         0.00
weekday_Friday            1.00
weekday_Monday            0.00
weekday_Saturday          0.00
weekday_Sunday            0.00
weekday_Tuesday           0.00
weekday_Wednesday         0.00
Name: 1, dtype: float64


In [242]:
rfc_predictions = rfc.predict(X)
df_true_vs_rfc_predicted = pd.DataFrame({'ActualClass': y, 'PredictedClass': rfc_predictions})
df_true_vs_rfc_predicted

ActualClass  PredictedClass
0              1               1
1              1               1
2              1               1
3              1               1
4              1               1
5              1               1
6              1               1
7              1               1
8              1               1
9              1               1
10             1               1
11             1               1
12             1               1
13             1               1
14             1               1
15             1               1
16             1               1
17             1               1
18             1               1
19             1               1
20             1               1
21             1               1
22             2               2
23             3               3
24             3               3
25             4               4
26             4               4
27             5               5
28             5               5
29             5               5
..           ...             ...
634           27              27
635           26              26
636           27              27
637           27              27
638           28              28
639           29              29
640           30              30
641           31              31
642           31              31
643           30              30
644           31              31
645           31              31
646           31              31
647           30              30
648           30              30
649           31              31
650           31              31
651           31              31
652           30              30
653           29              29
654           28              28
655           21              21
656           21              21
657           21              21
658           20              20
659           21              21
660           23              23
661           24              24
662           24              24
663           23              23

[664 rows x 2 columns]

In [243]:
print("Accuracy: ", metrics.accuracy_score(y, rfc_predictions))
print("Confusion matrix: \n", metrics.confusion_matrix(y, rfc_predictions))
print("Classification report:\n ", metrics.classification_report(y, rfc_predictions))

Accuracy:  1.0
Confusion matrix: 
 [[76  0  0 ...,  0  0  0]
 [ 0 29  0 ...,  0  0  0]
 [ 0  0 19 ...,  0  0  0]
 ..., 
 [ 0  0  0 ..., 20  0  0]
 [ 0  0  0 ...,  0 36  0]
 [ 0  0  0 ...,  0  0 83]]
Classification report:
               precision    recall  f1-score   support

          0       1.00      1.00      1.00        76
          1       1.00      1.00      1.00        29
          2       1.00      1.00      1.00        19
          3       1.00      1.00      1.00        14
          4       1.00      1.00      1.00        11
          5       1.00      1.00      1.00        12
          6       1.00      1.00      1.00        14
          7       1.00      1.00      1.00        13
          8       1.00      1.00      1.00        18
          9       1.00      1.00      1.00        24
         10       1.00      1.00      1.00        41
         11       1.00      1.00      1.00        17
         12       1.00      1.00      1.00        17
         13       1.00      1.00 

In [244]:
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=True, random_state=1, verbose=0, warm_start=False)

In [245]:
rfc_predictions_test = rfc.predict(X_test)
df_true_vs_rfc_predicted_test = pd.DataFrame({'ActualClass': y_test, 'PredictedClass': rfc_predictions_test})
df_true_vs_rfc_predicted_test

ActualClass  PredictedClass
166           10              10
297           22              22
286           20              19
116            8               6
631           26              26
144           18              16
599            6               5
89            27              24
506            6               7
650           31              31
41            10               9
460            4               3
43            10               9
192           12              11
603           10               9
353           22              22
242           28              29
58            29              29
583            8               9
130           13              12
565            0               0
310           27              27
88            29              31
320           20              19
482            0               0
81            30              31
143           18              16
83            31              30
282           18              21
170           10              10
..           ...             ...
616           14              12
149           19              20
32             7               8
137           17              13
572            1               0
36            14              15
306           29              29
303           28              29
398           29              30
527            1               2
459            5               5
261           30              31
442            6               5
593            9               9
66            30              31
301           27              24
358           24              23
523            1               0
335           19              19
610            9              11
205           22              22
104           10               9
113            6               4
151           19              20
8              1               1
178           10              10
547            0               0
634           27              26
319           20              19
597            4               2

[200 rows x 2 columns]

In [246]:
print("Accuracy: ", metrics.accuracy_score(y_test, rfc_predictions_test))
print("Confusion matrix: \n", metrics.confusion_matrix(y_test, rfc_predictions_test))
print("Classification report:\n ", metrics.classification_report(y_test, rfc_predictions_test))

Accuracy:  0.405
Confusion matrix: 
 [[15  2  2 ...,  0  0  0]
 [ 3  5  2 ...,  0  0  0]
 [ 0  1  3 ...,  0  0  0]
 ..., 
 [ 0  0  0 ...,  3  4  1]
 [ 0  0  0 ...,  2  1  6]
 [ 0  0  0 ...,  1  2 17]]
Classification report:
               precision    recall  f1-score   support

          0       0.83      0.79      0.81        19
          1       0.62      0.50      0.56        10
          2       0.38      0.50      0.43         6
          3       0.50      0.67      0.57         3
          4       0.00      0.00      0.00         4
          5       0.29      1.00      0.44         2
          6       0.00      0.00      0.00         5
          7       0.25      0.25      0.25         4
          8       0.00      0.00      0.00         7
          9       0.19      0.60      0.29         5
         10       0.88      0.44      0.58        16
         11       0.00      0.00      0.00         0
         12       0.25      0.17      0.20         6
         13       0.20      0.3

/Users/yaoxuanchen/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/yaoxuanchen/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [247]:
# compute the out-of-bag classification accuracy
rfc.oob_score_

0.54741379310344829

In [248]:
for i in range(10,110,10):
    print(i)
    rfc = RandomForestClassifier(n_estimators=i, max_features='auto', oob_score=True)
    rfc.fit(X_train, y_train)
    rfc_predictions_test = rfc.predict(X_test)
    print("Accuracy: ", metrics.accuracy_score(y_test, rfc_predictions_test))
    rfc.oob_score_

10
Accuracy:  0.41
20
Accuracy:  0.43
30
Accuracy:  0.385
40
Accuracy:  0.41

/Users/yaoxuanchen/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/yaoxuanchen/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])



50
Accuracy:  0.4
60
Accuracy:  0.41
70
Accuracy:  0.405
80
Accuracy:  0.395
90
Accuracy:  0.4
100
Accuracy:  0.415


In [258]:
cur = conn.cursor()
cur.execute('select * from station_1 ORDER BY last_update DESC limit 0, 1')
detailData = cur.fetchone()
detailData[0].minute

35

In [253]:
X.columns

Index(['humidity', 'temp', 'wind', 'day', 'hour', 'minute', 'pre_minute',
       'pre_available_bikes', 'weather_main_Clear', 'weather_main_Clouds',
       'weather_main_Drizzle', 'weather_main_Fog', 'weather_main_Mist',
       'weather_main_Rain', 'weekday_Friday', 'weekday_Monday',
       'weekday_Saturday', 'weekday_Sunday', 'weekday_Tuesday',
       'weekday_Wednesday'],
      dtype='object')

In [259]:
import json, requests
from datetime import datetime

r_w = requests.get('http://api.openweathermap.org/data/2.5/forecast?q=Dublin,IE&APPID=7b3e054e55cf81602b4298ca04a0fa18')

di = {x : 0 for x in sorted(X.columns)}

di['pre_available_bikes'] = detailData[-1]
di['pre_minute'] = detailData[0].hour * 60 + detailData[0].minute

di['day'] = datetime.utcfromtimestamp(r_w.json()['list'][0]['dt']).day
di['hour'] = datetime.utcfromtimestamp(r_w.json()['list'][0]['dt']).hour
di['humidity'] = r_w.json()['list'][0]['main']['humidity']
di['minute'] = datetime.utcfromtimestamp(r_w.json()['list'][0]['dt']).minute
di['temp'] = r_w.json()['list'][0]['main']['temp']
di['wind'] = r_w.json()['list'][0]['wind']['speed']

weather = r_w.json()['list'][0]['weather'][0]['main']
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday',
            'Friday', 'Saturday', 'Sunday']
weekday = weekdays[datetime.utcfromtimestamp(r_w.json()['list'][0]['dt']).weekday()]
if 'weather_main_{}'.format(weather) in di:
    di['weather_main_{}'.format(weather)] = 1
if 'weekday_{}'.format(weekday) in di:
    di['weekday_{}'.format(weekday)] = 1

In [260]:
predi = pd.DataFrame([di])
predi

day  hour  humidity  minute  pre_available_bikes  pre_minute    temp  \
0   18    15        76       0                   23         815  287.88   

   weather_main_Clear  weather_main_Clouds  weather_main_Drizzle  \
0                   1                    0                     0   

   weather_main_Fog  weather_main_Mist  weather_main_Rain  weekday_Friday  \
0                 0                  0                  0               0   

   weekday_Monday  weekday_Saturday  weekday_Sunday  weekday_Tuesday  \
0               0                 0               0                0   

   weekday_Wednesday   wind  
0                  1  12.21

In [261]:
rfc_predictions = rfc.predict(predi)
print(rfc_predictions)

[0]
